#### **Cell 1: ⬇️ Install Google GenAI Library**

This cell installs the necessary `GenAI` Python library to run Gemini.

In [ ]:
# Cell 1: Install the necessary minimum SDK version
print("Ensuring google-genai >= 1.51.0 is installed for Gemini 3 features...")
# We explicitly require version 1.51.0 or higher for full Gemini 3 support
!pip install -q 'google-genai>=1.51.0' pydantic
print("✅ Installation complete.")

Ensuring google-genai >= 1.51.0 is installed for Gemini 3 features...
✅ Installation complete.


In [ ]:
!pip show google-genai

Name: google-genai
Version: 1.51.0
Summary: GenAI Python SDK
Home-page: https://github.com/googleapis/python-genai
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Requires: anyio, google-auth, httpx, pydantic, requests, tenacity, typing-extensions, websockets
Required-by: google-adk, google-cloud-aiplatform


#### **Cell 2: ⚙️ Setup & Authentication**

This cell imports all required libraries and mounts your Google Drive so the notebook can access your files.

  * Ensure your Colab Secret is named `GOOGLE_API_KEY`.
  * When you run this cell, Google will ask for permission to access your Drive.

In [ ]:
# Cell 2: Imports & Client Initialization
import os
import json
import asyncio
from pathlib import Path
from typing import List, Optional, Literal

# New Google SDK imports
from google import genai
from google.genai import types
from google.colab import userdata, drive
from tqdm.asyncio import tqdm_asyncio
from pydantic import BaseModel, Field

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

# 2. Retrieve API Key from Secrets
try:
    API_KEY = userdata.get('GOOGLE_API_KEY')
except userdata.SecretNotFoundError:
    print("❌ Secret 'GOOGLE_API_KEY' not found. Please add it in the side panel.")
    raise

# 3. Initialize the Client and the Async Wrapper
client = genai.Client(api_key=API_KEY)
# 🎯 FIX: Explicitly define the async client wrapper for Cell 5 to use
async_client = client.aio

print("✅ Client initialized successfully. Asynchronous client ready.")

✅ Client initialized successfully. Asynchronous client ready.


#### **Cell 3: 🧠 Schema Definition ( The "Brain" )**

Defines the Structure. This is the most critical cell. By using Pydantic classes, we force Gemini 3 to output strict JSON matching this shape, eliminating the need for complex prompt engineering.

* Edit here if you need to add a new field (e.g., `cover_depth_mm`), add it to the relevant class below.

In [ ]:
# Cell 3: Define the Extraction Schema (PRESCRIPTIVE UPDATE)

# --- Sub-Components ---

class SpacingItem(BaseModel):
    # CRUCIAL: Standardizing Rto./Resto to 'rest'
    quantity: str = Field(..., description="The number of spaces (e.g. '10' from '10@100') or the verbatim string 'rest' for remaining length.")
    spacing: float = Field(..., description="The spacing distance in mm")

class StirrupDimensions(BaseModel):
    # Defining internal spans for 3D modeling
    span_width_mm: Optional[float] = Field(None, description="Internal clear width this tie or stirrup encloses/spans.")
    span_depth_mm: Optional[float] = Field(None, description="Internal clear depth this tie or stirrup encloses/spans.")

class TransverseReinforcement(BaseModel):
    stirrup_id: Optional[str] = Field(None, description="Verbatim ID/Name of the stirrup")
    stirrup_type: Literal["main_stirrup", "intermediate_stirrup", "internal_tie", "cross_tie"]
    bar_diameter_mm: Optional[float] = Field(None, description="Bar diameter in mm")
    # Using the updated StirrupDimensions class
    stirrup_dimensions: Optional[StirrupDimensions] = None
    stirrup_shape: Literal["rectangular", "circular", "L-shaped", "U-shaped", "diamond", "custom"]
    spacing_mm: List[SpacingItem] # Required
    # Removed location_description to avoid subjective text
    reference_code: Optional[str] = Field(None, description="Verbatim callout text")
    zone: Optional[str] = None


class LongitudinalReinforcement(BaseModel):
    bar_group_id: Optional[str] = Field(None, description="Identifier if bars are grouped")
    bar_diameter_mm: Optional[float] = Field(None, description="Bar diameter in mm")
    bar_count: int = Field(..., description="REQUIRED: The total number of bars.")
    reference_code: str = Field(..., description="REQUIRED: Verbatim callout text (e.g., '14Ø5/8')")
    zone: Optional[str] = None

    # NEW PRESCRIPTIVE FIELDS FOR BAR PLACEMENT
    bar_x_columns: int = Field(..., description="Total number of vertical columns (bar positions along the X-axis). E.g., 2, 4, 6.")
    bar_y_matrix: List[int] = Field(..., description="REQUIRED: List of bar counts for each vertical column defined by bar_x_columns. Sum must equal bar_count.")


class ConcreteSpecs(BaseModel):
    concrete_strength: str
    modulus_of_elasticity: Optional[str] = None
    clear_cover_mm: Optional[float] = Field(None, description="REQUIRED for 3D modeling, but extracted separately. Use the value provided by the agent if present.")


class Geometry(BaseModel):
    cross_section_type: Literal["rectangular", "circular", "L-shaped", "T-shaped"]
    width_mm: Optional[float] = None
    depth_mm: Optional[float] = None
    diameter_mm: Optional[float] = None

class ElementIdentification(BaseModel):
    type_of_element: str
    element_id: str
    level_reference: Optional[str] = None
    section_reference: Optional[str] = None
    scale: Optional[str] = None

class ReinforcementLayout(BaseModel):
    total_vertical_bars: Optional[int] = None
    total_stirrup_sets: Optional[int] = None
    reinforcement_pattern: Optional[str] = Field(None, description="Simplified description of the tie pattern, e.g., '3 Rectangular + 1 C-Tie'")

# --- Main Container ---
class ColumnExtraction(BaseModel):
    element_identification: ElementIdentification
    geometry: Geometry
    concrete_specifications: Optional[ConcreteSpecs] = None
    longitudinal_reinforcement: List[LongitudinalReinforcement]
    transverse_reinforcement: List[TransverseReinforcement]
    reinforcement_layout: ReinforcementLayout

#### **Cell 4: 🎛️ Configuration**

Sets the "Knobs and Dials." This includes input/output paths, the Model ID, and the System Prompt.
* Gemini 3 Settings: We configure thinking_level="HIGH" here.

In [ ]:
# Cell 4: Configuration

# --- Model Settings ---
# Using Gemini 3 Pro with High Thinking Level for maximum reasoning on complex CAD
MODEL_ID = "gemini-3.0-pro"

# Define the "Thinking" configuration
# Gemini 3 uses 'thinking_level' (HIGH/MEDIUM/LOW) instead of raw token budgets
THINKING_CONFIG = types.ThinkingConfig(
    include_thoughts=True, # Useful for debugging the logic
    thinking_level="HIGH"  # Maximize reasoning capabilities
)

# --- Path Settings ---
# Update these to match your exact Drive folder structure
DRIVE_BASE = Path("/content/drive/Shareddrives/Projects/04_Ingenium_edificio_jctello/screenshots/extraction_screenshots")
INPUT_DIR = DRIVE_BASE / "columns"
OUTPUT_DIR = DRIVE_BASE / "Columns_Outputs_Gemini3_v2"

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)
FILE_EXTENSIONS = [".png", ".jpg", ".jpeg"]

# --- System Prompt ---
# Note: We do NOT need to describe the JSON schema here. The SDK handles that.
# We only describe the BEHAVIOR and VISUAL INTERPRETATION rules.
SYSTEM_PROMPT = """
Role: You are a precise structural extractor and expert construction estimator, specializing in converting drawing graphics into geometric input data for 3D modeling.
Task: Analyze the column cross-section image and extract all technical and PRESCRIPTIVE specifications.

Critical Extraction Rules for 3D Modeling:
1. **Strict Observation:** Extract ALL visible information. Do not guess. Use null for missing values.
2. **Standardization:** All dimensions MUST be converted to **millimeters (mm)**.
3. **Verbatim:** For IDs, references, and concrete strength, copy the exact text.

**Prescriptive Rules for Reinforcement (Crucial for 3D Scripting):**
A. **Longitudinal Bar Placement (bar_x_columns & bar_y_matrix):**
    - Analyze the visual grid of the longitudinal bars.
    - 'bar_x_columns': Count the total number of vertical columns (along the W dimension, which is the shorter .42m side).
    - 'bar_y_matrix': Provide a list detailing the number of bars in each of those vertical columns (along the D dimension, the longer .70m side), starting from the left.
    *Example Image C-02 (42cm x 70cm): bar_x_columns=2, bar_y_matrix=[7, 7].* (4 corners + 5 intermediates = 7 per side)
B. **Transverse Tie Dimensions (span_width_mm / span_depth_mm):**
    - For all stirrups/ties, calculate the **internal clear dimension** they span (distance between the internal faces of the surrounding main reinforcement). Use the overall dimensions and standard cover rules if not explicit.
C. **Spacing Standardization:** For variable spacing indicators (e.g., "Rto." [Resto], "Rem.", or "Rest"), the **quantity** field in the spacing array MUST be the verbatim English word: **"rest"** (all lowercase).
D. **Required Fields:** Ensure 'bar_count', 'reference_code', 'stirrup_type', and 'spacing_mm' are never null.
"""

print(f"✅ Configuration Loaded.")
print(f"🤖 Model: {MODEL_ID} (Thinking Level: HIGH)")
print(f"📂 Input: {INPUT_DIR}")
print(f"📂 Output: {OUTPUT_DIR}")

✅ Configuration Loaded.
🤖 Model: gemini-3.0-pro (Thinking Level: HIGH)
📂 Input: /content/drive/Shareddrives/Projects/04_Ingenium_edificio_jctello/screenshots/extraction_screenshots/columns
📂 Output: /content/drive/Shareddrives/Projects/04_Ingenium_edificio_jctello/screenshots/extraction_screenshots/Columns_Outputs_Gemini3_v2


#### **Cell 4.5: Model Selection**
List available models and select the best one.

In [ ]:
# Cell 5: List Models and Select the Correct Alias
print("Listing available models to find a suitable ID...")

# 1. Fallback/Target Model IDs
target_models = [
    # 1. Try the most advanced preview alias (user's target)
    "models/gemini-3-pro-preview",
    # 2. Try the general preview alias
    "models/gemini-2.5-pro-preview",
    # 3. Fallback to the stable, high-capability model
    "models/gemini-2.5-pro"
]

selected_model_id = None
available_models = list(client.models.list())
available_names = [m.name for m in available_models]

# 2. Search for the best available model
for target in target_models:
    # Check if the target name (e.g., 'models/gemini-3-pro-preview') is in the list
    if target in available_names:
        selected_model_id = target.replace('models/', '')
        break

# 3. Handle cases where no target model is found (shouldn't happen with 2.5-pro)
if selected_model_id is None:
    print("❌ Critical Error: Could not find a suitable high-end model.")
    selected_model_id = "gemini-2.5-pro" # Ensure fallback for code execution

# 4. Final Configuration Update
MODEL_ID = selected_model_id

print(f"✅ Found and configured model ID: {MODEL_ID}")
print("Please run Cell 5 (the batch process) now.")

Listing available models to find a suitable ID...
✅ Found and configured model ID: gemini-3-pro-preview
Please run Cell 5 (the batch process) now.


#### **Cell 5: 🚀 Execution (The Engine)**
Runs the batch process. It looks for images, sends them to Gemini 3, parses the result, and saves the JSON.
* Async: Uses asyncio to process efficiently (though Thinking models may take longer per request, so we process them sequentially or with limited concurrency to avoid rate limits).

In [ ]:
# Cell 5: Asynchronous Batch Execution

async def process_single_image(image_path: Path):
    """Asynchronously processes a single image file."""
    output_filename = image_path.with_suffix(".json").name
    output_path = OUTPUT_DIR / output_filename

    # We use the async client ('async_client') here
    try:
        # 1. Load Image
        image_bytes = image_path.read_bytes()

        # 2. Prepare the Request Config
        config = types.GenerateContentConfig(
            system_instruction=SYSTEM_PROMPT,
            response_mime_type="application/json",
            response_schema=ColumnExtraction, # Enforces Pydantic Schema
            thinking_config=THINKING_CONFIG
        )

        # 3. Call the Async API
        response = await async_client.models.generate_content(
            model=MODEL_ID,
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_bytes(data=image_bytes, mime_type="image/png"),
                        types.Part.from_text(text="Extract the column specifications."),
                    ]
                )
            ],
            config=config
        )

        # 4. Parse & Save
        # The response.parsed property is automatically a Pydantic object
        result_dict = response.parsed.model_dump(mode='json', exclude_none=False)

        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(result_dict, f, indent=2)

        return f"✅ Success: {image_path.name}"

    except Exception as e:
        # Catch and report errors without stopping the entire batch
        return f"❌ Error {image_path.name}: {str(e)[:500]}..."

# --- Main Asynchronous Loop ---
async def run_batch():
    # Find all image files
    image_files = []
    for ext in FILE_EXTENSIONS:
        image_files.extend(INPUT_DIR.glob(f"*{ext}"))

    if not image_files:
        print(f"⚠️ No images found in {INPUT_DIR}")
        return

    print(f"Found {len(image_files)} images. Starting concurrent extraction...")

    # Create a list of async tasks
    tasks = [process_single_image(img) for img in image_files]

    # Run all tasks concurrently and display progress
    results = await tqdm_asyncio.gather(*tasks, desc="Processing images concurrently")

    print("\n--- Processing Report ---")
    for res in results:
        print(res)

# Execute the asynchronous batch
await run_batch()

Found 13 images. Starting concurrent extraction...


Processing images concurrently: 100%|██████████| 13/13 [00:55<00:00,  4.28s/it]


--- Processing Report ---
✅ Success: 05_09-00-58-00.png
✅ Success: 05_09-00-57-53.png
✅ Success: 05_09-00-58-17.png
✅ Success: 05_09-00-58-57.png
✅ Success: 05_09-00-58-30.png
✅ Success: 05_09-00-59-08.png
✅ Success: 05_09-00-58-36.png
✅ Success: 05_09-00-58-50.png
✅ Success: 05_09-00-59-20.png
✅ Success: 05_09-00-58-05.png
✅ Success: 05_09-00-58-24.png
✅ Success: 05_09-00-57-48.png
✅ Success: 05_09-00-58-43.png


#### **Cell 6: (Optional) 📄 View a Sample Result**

After running the batch, you can use this cell to load and view one of the JSON output files to confirm the data was extracted correctly.

In [ ]:
# Cell 6: (Optional) View a Sample Result

# List the files in your output directory
print(f"Files in {OUTPUT_DIR}:")
!ls -lh "{OUTPUT_DIR}"

# --- Manually set a filename to check ---
# (Change this to one of your output files)
filename_to_check = "05_09-00-58-00.json"

# ----------------------------------------

if filename_to_check:
    try:
        result_path = OUTPUT_DIR / filename_to_check
        with open(result_path, 'r') as f:
            data = json.load(f)
        print(f"\n--- Displaying content of {filename_to_check} ---")
        print(json.dumps(data, indent=2))
    except FileNotFoundError:
        print(f"\nError: File '{filename_to_check}' not found in {OUTPUT_DIR}")
    except Exception as e:
        print(f"An error occurred: {e}")
else:
    print("\nSet 'filename_to_check' to the name of a .json file to view its content.")

Files in /content/drive/Shareddrives/Projects/04_Ingenium_edificio_jctello/screenshots/extraction_screenshots/Columns_Outputs_Gemini3_v2:
total 28K
-rw------- 1 root root 1.5K Nov 18 19:37 05_09-00-57-48.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-57-53.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-58-00.json
-rw------- 1 root root 2.1K Nov 18 19:37 05_09-00-58-05.json
-rw------- 1 root root 2.2K Nov 18 19:37 05_09-00-58-17.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-58-24.json
-rw------- 1 root root 2.2K Nov 18 19:37 05_09-00-58-30.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-58-36.json
-rw------- 1 root root 2.2K Nov 18 19:37 05_09-00-58-43.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-58-50.json
-rw------- 1 root root 2.2K Nov 18 19:37 05_09-00-58-57.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-59-08.json
-rw------- 1 root root 1.6K Nov 18 19:37 05_09-00-59-20.json

--- Displaying content of 05_09-00-58-00.json ---
{
  "ele

#### **Cell 7: 🧮 3D Geometry Calculation Helper**

This cell defines the logic to convert prescriptive JSON data into a set of 3D coordinates.

In [ ]:
# Cell 7: 🧮 3D Geometry Calculation Helper & Data Loading**

import numpy as np
import math
from typing import List, Tuple
import os
from pathlib import Path

# --- Configuration (Must Match External/Assumed Values) ---
# NOTE: The script assumes these values for visualization purposes.
# In a true agentic pipeline, 'clear_cover_mm' and 'COLUMN_CLEAR_HEIGHT_MM'
# would be fed from upstream processes.

# We'll use the clear cover from the successful JSON output (40.0mm).
ASSUMED_CLEAR_COVER_MM = 40.0
# Assume a generic clear height for the Z-axis visualization (e.g., 3 meters)
COLUMN_CLEAR_HEIGHT_MM = 3000.0
# -----------------------------------------------------------

# --- 🎯 SET THE FILENAME HERE ---
# Use one of the JSON files successfully generated by the batch process (Cell 6)
filename_to_check = "05_09-00-58-00.json"
# ---------------------------------

# Ensure OUTPUT_DIR is defined (assuming Cell 4 has been run)
try:
    if 'OUTPUT_DIR' not in locals():
        raise NameError("OUTPUT_DIR not found. Please run Cell 4 first.")

    result_path = Path(OUTPUT_DIR) / filename_to_check
    with open(result_path, 'r') as f:
        json_data = json.load(f)

    print(f"✅ Successfully loaded data from: {filename_to_check}")

except NameError as e:
    print(f"❌ Error: {e}")
    json_data = None # Stop execution
except FileNotFoundError:
    print(f"❌ Error: File '{filename_to_check}' not found in {OUTPUT_DIR}. Check the name and directory.")
    json_data = None
except Exception as e:
    print(f"❌ Error loading JSON: {e}")
    json_data = None


def calculate_bar_coordinates(json_data: dict) -> Tuple[List[float], List[float], List[float], float]:
    """
    Calculates X, Y, and Z coordinates for all longitudinal bars based on the
    prescriptive fields in the JSON output.
    """
    if not json_data:
        return [], [], [], 0.0

    # 1. Retrieve Core Geometric Data
    W = json_data['geometry']['width_mm']
    D = json_data['geometry']['depth_mm']
    cover = json_data['concrete_specifications'].get('clear_cover_mm', ASSUMED_CLEAR_COVER_MM) # Default to value set above if missing

    long_data = json_data['longitudinal_reinforcement'][0]
    bar_x_cols = long_data.get('bar_x_columns', 2)
    bar_y_matrix = long_data.get('bar_y_matrix', [long_data['bar_count']//2, long_data['bar_count']//2])

    # 2. Calculate Effective Dimensions and Spacing
    W_effective = W - 2 * cover
    D_effective = D - 2 * cover

    # Calculate bar positions along the W (X) axis
    x_positions = np.linspace(cover, W - cover, bar_x_cols)

    # 3. Generate Coordinates (X, Y)
    all_x = []
    all_y = []

    for col_index, num_bars_in_col in enumerate(bar_y_matrix):
        if num_bars_in_col > 1:
            # Multi-bar column: bars are spaced vertically (Y-axis)
            y_coords = np.linspace(cover, D - cover, num_bars_in_col)
        elif num_bars_in_col == 1:
            # Single bar column: bar is centered vertically
            y_coords = [D / 2]
        else:
            continue

        current_x = x_positions[col_index]
        all_x.extend([current_x] * num_bars_in_col)
        all_y.extend(y_coords)

    # 4. Generate Z Coordinates
    all_z = [COLUMN_CLEAR_HEIGHT_MM / 2] * len(all_x)

    return all_x, all_y, all_z, cover

# Execute calculation if data was loaded
if json_data:
    bar_coords_x, bar_coords_y, bar_coords_z, clear_cover_used = calculate_bar_coordinates(json_data)
    print("✅ Coordinates calculated and ready for visualization.")

✅ Successfully loaded data from: 05_09-00-58-00.json
✅ Coordinates calculated and ready for visualization.


#### **Cell 8: 📊 3D Visualization of Reinforcement**
This cell generates an interactive 3D plot of the column cross-section.

In [ ]:
## *Cell 8: 📊 3D Visualization of Reinforcement**

import plotly.graph_objects as go
import pandas as pd
from IPython.display import display, Markdown

if 'json_data' not in locals() or not json_data:
    print("Cannot run visualization: Data not loaded in Cell 7.")
else:
    # 1. Prepare Core Data
    W = json_data['geometry']['width_mm']
    D = json_data['geometry']['depth_mm']

    long_bar_data = json_data['longitudinal_reinforcement'][0]
    bar_size = f"#{long_bar_data['bar_diameter_mm']}mm"

    # 2. Create DataFrame for Plotly
    df = pd.DataFrame({
        'X': bar_coords_x,
        'Y': bar_coords_y,
        'Z': bar_coords_z,
        'Bar_Size': [bar_size] * len(bar_coords_x),
    })

    # 3. Define the Concrete Envelope (for visual context)
    concrete_corners_x = [0, W, W, 0, 0]
    concrete_corners_y = [0, 0, D, D, 0]
    concrete_corners_z = [bar_coords_z[0]] * 5

    # 4. Create the Plotly Figure

    # Trace 1: Longitudinal Bars (Scatter plot)
    bar_trace = go.Scatter3d(
        x=df['X'],
        y=df['Y'],
        z=df['Z'],
        mode='markers',
        marker=dict(
            size=10,
            color='red',
            symbol='circle',
            opacity=0.8
        ),
        name=f"Longitudinal Bars ({long_bar_data['bar_count']}x {bar_size})",
        hovertext=[f"Bar @ ({x:.0f}mm, {y:.0f}mm)" for x, y in zip(df['X'], df['Y'])],
        hoverinfo='text'
    )

    # Trace 2: Concrete Perimeter (Line trace)
    perimeter_trace = go.Scatter3d(
        x=concrete_corners_x,
        y=concrete_corners_y,
        z=concrete_corners_z,
        mode='lines',
        line=dict(color='gray', width=3),
        name='Concrete Perimeter (W x D)'
    )

    fig = go.Figure(data=[bar_trace, perimeter_trace])

    # 5. Configure Layout
    fig.update_layout(
        scene=dict(
            xaxis_title=f"Width (X) - {W}mm",
            yaxis_title=f"Depth (Y) - {D}mm",
            zaxis_title="Z-Axis (Center Plane)",
            aspectmode='data',
            camera=dict(
                up=dict(x=0, y=0, z=1),
                center=dict(x=0, y=0, z=0),
                eye=dict(x=1.5, y=1.5, z=1.5)
            )
        ),
        title=f"3D Reinforcement Cross-Section: {data['element_identification']['element_id']}",
        margin=dict(l=0, r=0, b=0, t=40)
    )

    # 6. Display
    display(Markdown("## 🏗️ Interactive 3D Cross-Section View"))
    display(Markdown(f"**Element:** {data['element_identification']['element_id']} | **Size:** {W}mm x {D}mm | **Cover Used:** {clear_cover_used}mm"))
    fig.show()

    print("\nVisualization complete. The bar placement is derived deterministically from the 'bar_x_columns' and 'bar_y_matrix' prescriptive fields.")

## 🏗️ Interactive 3D Cross-Section View

**Element:** C-02 | **Size:** 420.0mm x 700.0mm | **Cover Used:** 40.0mm


Visualization complete. The bar placement is derived deterministically from the 'bar_x_columns' and 'bar_y_matrix' prescriptive fields.
